<a href="https://colab.research.google.com/github/MrWou/Emilia_Romagna_Museum_data/blob/main/SPARQL_MUSEI_E_CULTURA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CODE PER CHIAMATA A ENDPOINT SPARQL MUSEI DELL'EMILIA ROMAGNA
Il codice recupera le informazioni disponibili sui Musei dell'Emilia-Romagna catalogati da

*Parametri richiesti:*

'museo',<br>
'nome', <br>
'latitudine', <br>
'longitudine', <br>
'ubicazione',<br>
'nome_comune',<br>
'descrizione_it',<br>
'descrizione_en',<br>
'sameAs_beniculturali_link',<br>
'sameAs_wikidata_link',<br>
'givesAdmission_link_list', <br>
'givesAdmission_desc_list',<br>
'is_free_admission_list', <br>
'accessibility_list', <br>
'situated_link_list',<br>
'situated_sede_list', <br>
'street_address_list', <br>
'postal_code_list',<br>
'homepage_list', <br>
'purpose_link_list',<br>
'purpose_desc_list',<br>
'type_link_list', <br>
'includesArea_list', <br>
'offersService_list',<br>
'opening_hours_list'

In [ ]:
import pandas as pd
import requests
import io

#Definisci endpoint SPAQL
sparql_endpoint = 'https://dati.emilia-romagna.it/sparql'

#Definisci la QUERY SPARQL
sparql_query = """
PREFIX culturalis: <http://culturalis.org/cult/0.1#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ocsa: <http://culturalis.org/ocsa/1.0#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX gr: <http://purl.org/goodrelations/v1#>

SELECT
  ?museo
  ?nome
  ?latitudine
  ?longitudine
  ?ubicazione
  ?nome_comune
  ?descrizione_it
  ?descrizione_en
  (MIN(?sameAs_beniculturali) AS ?sameAs_beniculturali_link)
  (MIN(?sameAs_wikidata) AS ?sameAs_wikidata_link)
  (GROUP_CONCAT(DISTINCT ?givesAdmission_link; SEPARATOR=", ") AS ?givesAdmission_link_list)
  (GROUP_CONCAT(DISTINCT ?givesAdmission_desc; SEPARATOR="; ") AS ?givesAdmission_desc_list)
  (GROUP_CONCAT(DISTINCT STR(?is_free_admission); SEPARATOR=", ") AS ?is_free_admission_list)
  (GROUP_CONCAT(DISTINCT ?accessibility; SEPARATOR=", ") AS ?accessibility_list)
  (GROUP_CONCAT(DISTINCT ?situated_link; SEPARATOR=", ") AS ?situated_link_list)
  (GROUP_CONCAT(DISTINCT ?situated_sede; SEPARATOR="; ") AS ?situated_sede_list)
  (GROUP_CONCAT(DISTINCT ?street_address; SEPARATOR="; ") AS ?street_address_list)
  (GROUP_CONCAT(DISTINCT ?postal_code; SEPARATOR=", ") AS ?postal_code_list)
  (GROUP_CONCAT(DISTINCT ?homepage; SEPARATOR=", ") AS ?homepage_list)
  (GROUP_CONCAT(DISTINCT ?purpose_link; SEPARATOR=", ") AS ?purpose_link_list)
  (GROUP_CONCAT(DISTINCT ?purpose_desc; SEPARATOR="; ") AS ?purpose_desc_list)
  (GROUP_CONCAT(DISTINCT ?type_link; SEPARATOR=", ") AS ?type_link_list)
  (GROUP_CONCAT(DISTINCT ?includesArea_label; SEPARATOR="; ") AS ?includesArea_list)
  (GROUP_CONCAT(DISTINCT ?offersService_label; SEPARATOR="; ") AS ?offersService_list)
  (GROUP_CONCAT(DISTINCT ?opening_hours_label; SEPARATOR=" | ") AS ?opening_hours_list)
WHERE {
  ?museo a culturalis:Museum ;
         rdfs:label ?nome ;
         geo:lat ?latitudine ;
         geo:long ?longitudine ;
         geonames:locatedIn ?ubicazione .

  ?ubicazione rdfs:label ?nome_comune .

  OPTIONAL {
    ?museo dc:description ?descrizione_it .
    FILTER(LANG(?descrizione_it) = "it")
  }
  OPTIONAL {
    ?museo dc:description ?descrizione_en .
    FILTER(LANG(?descrizione_en) = "en")
  }

  OPTIONAL { ?museo owl:sameAs ?sameAs_beniculturali .
             FILTER (strstarts(str(?sameAs_beniculturali), "http://dati.beniculturali.it")) }
  OPTIONAL { ?museo owl:sameAs ?sameAs_wikidata .
             FILTER (strstarts(str(?sameAs_wikidata), "https://www.wikidata.org")) }

  OPTIONAL {
    ?museo ocsa:givesAdmission ?givesAdmission_link .
    OPTIONAL { ?givesAdmission_link rdfs:label ?givesAdmission_desc . }
    OPTIONAL { ?givesAdmission_link ocsa:accessibility ?accessibility . }
    OPTIONAL { ?givesAdmission_link ocsa:freeAdmission ?is_free_admission . }
  }

  OPTIONAL {
    ?museo culturalis:situated ?situated_link .
    OPTIONAL { ?situated_link rdfs:label ?situated_sede . }
    OPTIONAL {
      ?situated_link ocsa:hasAddress ?address_node .
      OPTIONAL { ?address_node vcard:street-address ?street_address . }
      OPTIONAL { ?address_node geo:postalCode ?postal_code . }
    }
  }

  OPTIONAL { ?museo foaf:homepage ?homepage . }

  OPTIONAL {
    ?museo org:purpose ?purpose_link .
    OPTIONAL { ?purpose_link rdfs:label ?purpose_desc . }
  }

  OPTIONAL { ?museo dcterms:type ?type_link . }

  OPTIONAL {
    ?museo ocsa:includesArea ?includesArea .
    ?includesArea rdfs:label ?includesArea_label .
  }

  OPTIONAL {
    ?museo ocsa:offersService ?offersService .
    ?offersService rdfs:label ?offersService_label .
  }

  OPTIONAL {
    ?museo ocsa:hasConditionOfUse ?condition_node .
    ?condition_node gr:hasOpeningHoursSpecification ?opening_hours_spec .
    OPTIONAL { ?opening_hours_spec rdfs:label ?opening_hours_label . }
  }

}
GROUP BY
  ?museo ?nome ?latitudine ?longitudine ?ubicazione ?nome_comune ?descrizione_it ?descrizione_en
"""

# Definisci gli header per la richiesta HTTP
# Richiediamo i risultati in formato CSV per facilitare la lettura con Pandas
# In alternativa si può usare 'application/sparql-results+json' e poi processare il JSON
headers = {
    'Accept': 'text/csv'
    # 'Accept': 'application/sparql-results+json' # Alternativa JSON
}

# Definisci i parametri per la richiesta POST
params = {
    'query': sparql_query
}

# Esegui la richiesta POST
try:
    response = requests.post(sparql_endpoint, data=params, headers=headers, timeout=60) # Timeout di 60 secondi
    response.raise_for_status() # Solleva un'eccezione per errori HTTP (es. 404, 500)

    # Controlla se la risposta è vuota o non valida prima di tentare di creare il DataFrame
    if response.text.strip():
         # Leggi i dati CSV direttamente dalla risposta in un DataFrame Pandas
        csv_data = io.StringIO(response.text)
        df = pd.read_csv(csv_data)

        # Mostra il DataFrame (o le prime righe)
        print("Query eseguita con successo. Prime 10 righe del DataFrame:")
        print(df.head(10).to_string()) # .to_string() per vedere tutte le colonne

        # Puoi anche mostrare informazioni sul DataFrame
        print("\nInformazioni sul DataFrame:")
        df.info()

        # Esempio: salvare il DataFrame completo in un file CSV
        # try:
        #     df.to_csv("risultati_musei_emilia_romagna.csv", index=False, encoding='utf-8-sig')
        #     print("\nDataFrame salvato come 'risultati_musei_emilia_romagna.csv'")
        # except Exception as e:
        #     print(f"\nErrore durante il salvataggio del CSV: {e}")

    else:
        print("La risposta dal server SPARQL è vuota.")


except requests.exceptions.RequestException as e:
    print(f"Errore durante la richiesta HTTP: {e}")
    # Se l'errore è dovuto a un timeout o a un problema del server, potresti vedere dettagli qui
    if hasattr(e, 'response') and e.response is not None:
         print("Dettagli errore dal server:")
         print(f"Status Code: {e.response.status_code}")
         print(f"Response Text: {e.response.text[:500]}...") # Stampa i primi 500 caratteri della risposta del server

except Exception as e:
    print(f"Si è verificato un errore generico: {e}")


Query eseguita con successo. Prime 10 righe del DataFrame:
                                              museo                                                                                           nome  latitudine  longitudine                                                         ubicazione          nome_comune                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

##Salva su CSV il Risultato

In [ ]:
df.to_csv("risultati_musei_emilia_romagna.csv", index=False, encoding='utf-8-sig')

##QUERY PER TEATRI STORICI

In [ ]:
import pandas as pd
import requests
import io

#Definisci endpoint SPAQL
sparql_endpoint = 'https://dati.emilia-romagna.it/sparql'

#Definisci la QUERY SPARQL
sparql_query = """
PREFIX culturalis: <http://culturalis.org/cult/0.1#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ocsa: <http://culturalis.org/ocsa/1.0#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX gr: <http://purl.org/goodrelations/v1#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dbpedia: <http://dbpedia.org/resource/>

SELECT
  ?teatro_storico
  ?nome
  ?latitudine
  ?longitudine
  ?ubicazione
  ?nome_comune
  ?descrizione_it
  ?descrizione_en
  (MIN(?sameAs_beniculturali) AS ?sameAs_beniculturali_link)
  (MIN(?sameAs_wikidata) AS ?sameAs_wikidata_link)
  (GROUP_CONCAT(DISTINCT ?givesAdmission_link; SEPARATOR=", ") AS ?givesAdmission_link_list)
  (GROUP_CONCAT(DISTINCT ?givesAdmission_desc; SEPARATOR="; ") AS ?givesAdmission_desc_list)
  (GROUP_CONCAT(DISTINCT STR(?is_free_admission); SEPARATOR=", ") AS ?is_free_admission_list)
  (GROUP_CONCAT(DISTINCT ?accessibility; SEPARATOR=", ") AS ?accessibility_list)
  (GROUP_CONCAT(DISTINCT ?situated_link; SEPARATOR=", ") AS ?situated_link_list)
  (GROUP_CONCAT(DISTINCT ?situated_sede; SEPARATOR="; ") AS ?situated_sede_list)
  (GROUP_CONCAT(DISTINCT ?street_address; SEPARATOR="; ") AS ?street_address_list)
  (GROUP_CONCAT(DISTINCT ?postal_code; SEPARATOR=", ") AS ?postal_code_list)
  (GROUP_CONCAT(DISTINCT ?homepage; SEPARATOR=", ") AS ?homepage_list)
  (GROUP_CONCAT(DISTINCT ?purpose_link; SEPARATOR=", ") AS ?purpose_link_list)
  (GROUP_CONCAT(DISTINCT ?purpose_desc; SEPARATOR="; ") AS ?purpose_desc_list)
  (GROUP_CONCAT(DISTINCT ?type_link; SEPARATOR=", ") AS ?type_link_list)
  (GROUP_CONCAT(DISTINCT ?includesArea_label; SEPARATOR="; ") AS ?includesArea_list)
  (GROUP_CONCAT(DISTINCT ?offersService_label; SEPARATOR="; ") AS ?offersService_list)
  (GROUP_CONCAT(DISTINCT ?opening_hours_label; SEPARATOR=" | ") AS ?opening_hours_list)
WHERE {
  # Filtra per istituzioni culturali che sono Teatri storici ed escludi i castelli
  ?teatro_storico a culturalis:CulturalInstitutionOrSite ;
                  rdfs:label ?nome ;
                  dcterms:source ?dataset .
  ?dataset skos:prefLabel "Teatri storici" .

  MINUS { ?teatro_storico a dbpedia:Castle }

  # Recupera le coordinate geografiche
  OPTIONAL {
    ?teatro_storico geo:lat ?latitudine ;
                    geo:long ?longitudine .
  }

  # Recupera l'ubicazione (link alla risorsa del comune/luogo) e il nome del comune
  OPTIONAL {
    ?teatro_storico geonames:locatedIn ?ubicazione .
    OPTIONAL { ?ubicazione rdfs:label ?nome_comune . }
  }

  # Recupera le descrizioni in italiano e inglese (se disponibili)
  OPTIONAL {
    ?teatro_storico dc:description ?descrizione_it .
    FILTER(LANG(?descrizione_it) = "it")
  }
  OPTIONAL {
    ?teatro_storico dc:description ?descrizione_en .
    FILTER(LANG(?descrizione_en) = "en")
  }

  # Recupera i link owl:sameAs a Beniculturali e Wikidata
  OPTIONAL {
    ?teatro_storico owl:sameAs ?sameAs_beniculturali .
    FILTER (strstarts(str(?sameAs_beniculturali), "http://dati.beniculturali.it"))
  }
  OPTIONAL {
    ?teatro_storico owl:sameAs ?sameAs_wikidata .
    FILTER (strstarts(str(?sameAs_wikidata), "https://www.wikidata.org"))
  }

  # Recupera informazioni sull'ammissione (ingressi)
  OPTIONAL {
    ?teatro_storico ocsa:givesAdmission ?givesAdmission_link .
    OPTIONAL { ?givesAdmission_link rdfs:label ?givesAdmission_desc . }
    OPTIONAL { ?givesAdmission_link ocsa:accessibility ?accessibility . } # Accessibilità legata all'ammissione
    OPTIONAL { ?givesAdmission_link ocsa:freeAdmission ?is_free_admission . } # Indica se l'ammissione è gratuita
  }

  # Recupera informazioni sulla sede fisica
  OPTIONAL {
    ?teatro_storico culturalis:situated ?situated_link .
    OPTIONAL { ?situated_link rdfs:label ?situated_sede . }
    OPTIONAL {
      ?situated_link ocsa:hasAddress ?address_node . # Nodo che rappresenta l'indirizzo
      OPTIONAL { ?address_node vcard:street-address ?street_address . } # Indirizzo via/piazza
      OPTIONAL { ?address_node geo:postalCode ?postal_code . } # Codice postale
    }
  }

  # Recupera la homepage ufficiale
  OPTIONAL { ?teatro_storico foaf:homepage ?homepage . }

  # Recupera lo scopo dell'organizzazione (potrebbe non essere popolato per singoli teatri)
  OPTIONAL {
    ?teatro_storico org:purpose ?purpose_link .
    OPTIONAL { ?purpose_link rdfs:label ?purpose_desc . }
  }

  # Recupera il tipo (potrebbe essere più granulare di culturalis:CulturalInstitutionOrSite)
  OPTIONAL { ?teatro_storico dcterms:type ?type_link . }

  # Recupera le aree incluse (più comune per musei, ma incluso come richiesto)
  OPTIONAL {
    ?teatro_storico ocsa:includesArea ?includesArea .
    OPTIONAL { ?includesArea rdfs:label ?includesArea_label . }
  }

  # Recupera i servizi offerti (più comune per musei, ma incluso come richiesto)
  OPTIONAL {
    ?teatro_storico ocsa:offersService ?offersService .
    OPTIONAL { ?offersService rdfs:label ?offersService_label . }
  }

  # Recupera gli orari di apertura (tramite ConditionOfUse e OpeningHoursSpecification)
  OPTIONAL {
    ?teatro_storico ocsa:hasConditionOfUse ?condition_node .
    ?condition_node gr:hasOpeningHoursSpecification ?opening_hours_spec .
    OPTIONAL { ?opening_hours_spec rdfs:label ?opening_hours_label . }
  }

}
GROUP BY
  ?teatro_storico ?nome ?latitudine ?longitudine ?ubicazione ?nome_comune ?descrizione_it ?descrizione_en
"""

# Definisci gli header per la richiesta HTTP
# Richiediamo i risultati in formato CSV per facilitare la lettura con Pandas
# In alternativa si può usare 'application/sparql-results+json' e poi processare il JSON
headers = {
    'Accept': 'text/csv'
    # 'Accept': 'application/sparql-results+json' # Alternativa JSON
}

# Definisci i parametri per la richiesta POST
params = {
    'query': sparql_query
}

# Esegui la richiesta POST
try:
    response = requests.post(sparql_endpoint, data=params, headers=headers, timeout=60) # Timeout di 60 secondi
    response.raise_for_status() # Solleva un'eccezione per errori HTTP (es. 404, 500)

    # Controlla se la risposta è vuota o non valida prima di tentare di creare il DataFrame
    if response.text.strip():
         # Leggi i dati CSV direttamente dalla risposta in un DataFrame Pandas
        csv_data = io.StringIO(response.text)
        df = pd.read_csv(csv_data)

        # Mostra il DataFrame (o le prime righe)
        print("Query eseguita con successo. Prime 10 righe del DataFrame:")
        print(df.head(10).to_string()) # .to_string() per vedere tutte le colonne

        # Puoi anche mostrare informazioni sul DataFrame
        print("\nInformazioni sul DataFrame:")
        df.info()

        # Esempio: salvare il DataFrame completo in un file CSV
        # try:
        #     df.to_csv("risultati_musei_emilia_romagna.csv", index=False, encoding='utf-8-sig')
        #     print("\nDataFrame salvato come 'risultati_musei_emilia_romagna.csv'")
        # except Exception as e:
        #     print(f"\nErrore durante il salvataggio del CSV: {e}")

    else:
        print("La risposta dal server SPARQL è vuota.")


except requests.exceptions.RequestException as e:
    print(f"Errore durante la richiesta HTTP: {e}")
    # Se l'errore è dovuto a un timeout o a un problema del server, potresti vedere dettagli qui
    if hasattr(e, 'response') and e.response is not None:
         print("Dettagli errore dal server:")
         print(f"Status Code: {e.response.status_code}")
         print(f"Response Text: {e.response.text[:500]}...") # Stampa i primi 500 caratteri della risposta del server

except Exception as e:
    print(f"Si è verificato un errore generico: {e}")
df.to_csv("risultati_teatri_storici_emilia_romagna.csv", index=False, encoding='utf-8-sig')

Query eseguita con successo. Prime 10 righe del DataFrame:
                                     teatro_storico                                                   nome  latitudine  longitudine                                                            ubicazione                nome_comune                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

##QUERY PER ARCHEO



In [ ]:
import pandas as pd
import requests
import io

#Definisci endpoint SPAQL
sparql_endpoint = 'https://dati.emilia-romagna.it/sparql'

#Definisci la QUERY SPARQL
sparql_query = """
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX clv: <https://w3id.org/italia/onto/CLV/>
PREFIX wgs84: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> # Usato per dc:date e dc:type
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> # Necessario per 'a' (rdf:type)
PREFIX core: <https://w3id.org/arco/ontology/core/>
PREFIX a-loc: <https://w3id.org/arco/ontology/location/>

SELECT DISTINCT
  ?site
  ?label
  ?cityName
  ?lat
  ?lon
  ?description
  ?addressResource
  ?addressLabel
  (GROUP_CONCAT(DISTINCT ?seeAlso; SEPARATOR=", ") AS ?seeAlso_links)
  (GROUP_CONCAT(DISTINCT ?date; SEPARATOR=", ") AS ?dates)
  (GROUP_CONCAT(DISTINCT ?dc_type_uri; SEPARATOR=", ") AS ?dc_types_uris) # Ora aggreghiamo gli URI dei tipi specifici (dc:type)
  (GROUP_CONCAT(DISTINCT ?fullAddress; SEPARATOR="; ") AS ?full_addresses)
  (GROUP_CONCAT(DISTINCT ?postalCode; SEPARATOR=", ") AS ?postal_codes)

WHERE {
  ?site a arco:ArchaeologicalProperty ; # Manteniamo questa restrizione
        rdfs:label ?label .

  # --- Informazioni di Posizione ---
  OPTIONAL {
     ?site geonames:locatedIn ?cityResource .
     OPTIONAL { ?cityResource rdfs:label ?cityName . }
  }
  OPTIONAL {
     ?site clv:hasGeometry ?geometry .
     OPTIONAL {
        ?geometry wgs84:lat ?lat ;
                  wgs84:long ?lon .
     }
  }
  OPTIONAL {
     ?site a-loc:hasCulturalPropertyAddress ?addressResource .
     OPTIONAL { ?addressResource rdfs:label ?addressLabel . }
     OPTIONAL { ?addressResource clv:fullAddress ?fullAddress . }
     OPTIONAL { ?addressResource vcard:postal-code ?postalCode . }
  }

  # --- Altre Informazioni Richieste ---
  OPTIONAL { ?site core:description ?description . }
  OPTIONAL { ?site rdfs:seeAlso ?seeAlso . }
  OPTIONAL { ?site dc:date ?date . }

  # --- Recupera gli URI dei valori collegati tramite dc:type ---
  OPTIONAL {
     ?site dc:type ?dc_type_uri . # Trova e recupera l'URI collegato tramite dc:type
     # Rimossa la parte che recuperava il label rdfs:label
  }

}
GROUP BY
  ?site
  ?label
  ?cityName
  ?lat
  ?lon
  ?description
  ?addressResource
  ?addressLabel
# Il GROUP BY non cambia

ORDER BY
  ?label
"""

# Definisci gli header per la richiesta HTTP
# Richiediamo i risultati in formato CSV per facilitare la lettura con Pandas
# In alternativa si può usare 'application/sparql-results+json' e poi processare il JSON
headers = {
    'Accept': 'text/csv'
    # 'Accept': 'application/sparql-results+json' # Alternativa JSON
}

# Definisci i parametri per la richiesta POST
params = {
    'query': sparql_query
}

# Esegui la richiesta POST
try:
    response = requests.post(sparql_endpoint, data=params, headers=headers, timeout=60) # Timeout di 60 secondi
    response.raise_for_status() # Solleva un'eccezione per errori HTTP (es. 404, 500)

    # Controlla se la risposta è vuota o non valida prima di tentare di creare il DataFrame
    if response.text.strip():
         # Leggi i dati CSV direttamente dalla risposta in un DataFrame Pandas
        csv_data = io.StringIO(response.text)
        df = pd.read_csv(csv_data)

        # Mostra il DataFrame (o le prime righe)
        print("Query eseguita con successo. Prime 10 righe del DataFrame:")
        print(df.head(10).to_string()) # .to_string() per vedere tutte le colonne

        # Puoi anche mostrare informazioni sul DataFrame
        print("\nInformazioni sul DataFrame:")
        df.info()

        # Esempio: salvare il DataFrame completo in un file CSV
        # try:
        #     df.to_csv("risultati_musei_emilia_romagna.csv", index=False, encoding='utf-8-sig')
        #     print("\nDataFrame salvato come 'risultati_musei_emilia_romagna.csv'")
        # except Exception as e:
        #     print(f"\nErrore durante il salvataggio del CSV: {e}")

    else:
        print("La risposta dal server SPARQL è vuota.")


except requests.exceptions.RequestException as e:
    print(f"Errore durante la richiesta HTTP: {e}")
    # Se l'errore è dovuto a un timeout o a un problema del server, potresti vedere dettagli qui
    if hasattr(e, 'response') and e.response is not None:
         print("Dettagli errore dal server:")
         print(f"Status Code: {e.response.status_code}")
         print(f"Response Text: {e.response.text[:500]}...") # Stampa i primi 500 caratteri della risposta del server

except Exception as e:
    print(f"Si è verificato un errore generico: {e}")
df.to_csv("risultati_aree_archeologiche_emilia_romagna.csv", index=False, encoding='utf-8-sig')

Query eseguita con successo. Prime 10 righe del DataFrame:
                                                                 site                                                         label     cityName  lat  lon                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   